# Lecture 4: Password CPA Attack - Attack Principle

In this example we want to improve the password check again to be resistant against the attack from the last tutorial.

## Improving the code

Let's first recap the password checking loop from the last lecture:
```c
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    if (stored_password[i] != passwd[i])
    {
        password_wrong = 1;
    }
}
```

The differences attack discussed in the last example worked because of the different power consumption when executing the code inside the if clause. This is addressed by the following code.

```c
uint8_t password_wrong = 0;
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    password_wrong |= stored_password[i] ^ passwd[i];
}
```

This is an excerpt from `4_password_fixed.c`.

In [ ]:
import securec
from securec import util
scope, target = util.init()

In [ ]:
securec.util.compile_and_flash('./4_password_fixed.c')

In [ ]:
import struct
import time
import warnings
import numpy as np

scope.default_setup()
scope.adc.samples = 500

def capture(attempt, count=1):
    if isinstance(attempt, str):
        attempt = attempt.encode('iso-8859-1')
    elif isinstance(attempt, int):
        attempt = bytes([attempt])
    traces = []
    for _ in range(count):
        scope.arm()
        target.simpleserial_write('p', attempt + b'\x00' * (10 - len(attempt)))
        result = target.simpleserial_read('p', 1)
        traces.append(util.capture())
    return np.mean(np.array(traces), axis=0), not bool(result[0])

In [ ]:
import math
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool
from bokeh.palettes import Category10_10, Turbo256, Inferno256
from bokeh.models import Span, Label, BoxAnnotation
from bokeh.layouts import column, row

output_notebook()

## From analysis to attack

In [ ]:
import numpy as np

HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    if isinstance(n, str):
        return HW[ord(n)]
    return HW[n]

hw_vec = np.vectorize(hw)

In [ ]:
import random
import tqdm
import tqdm.notebook

traces = []
for _ in tqdm.notebook.tqdm(range(500)):
    attempt = random.randint(0, 255)
    traces.append((attempt, capture(attempt)[0]))

### Displaying Hamming Weights

In the last notebook we visualized the relation between the hamming weight of an attempt and the value of its corresponding trace. 

In [ ]:
p = figure(title='hw(attempt) vs trace @ ld r18, X+', sizing_mode='stretch_width', plot_height=200)
p.toolbar_location = None
for attempt, trace in traces:
    p.circle(
        hw(attempt),
        trace[37],
        color=Category10_10[hw(attempt)],
        size=5,
    )

show(p)

The figure looks quite different if we look at the "wrong" position:

In [ ]:
p = figure(title='hw(attempt) vs trace @ eor r18, r24', sizing_mode='stretch_width', plot_height=200)
p.toolbar_location = None
for attempt, trace in traces:
    p.circle(
        hw(attempt),
        trace[45],
        line_color=Category10_10[hw(attempt)],
    )

show(p)

On position `eor r18, r24` the relationship between `hw(attempt)` and the trace is not linear. But there is a linear relationship if we modify the x-coordinate:

In [ ]:
p = figure(title='hw(attempt ^ i) vs trace @ eor r18, r24', sizing_mode='stretch_width', plot_height=200)
p.toolbar_location = None
for attempt, trace in traces:
    p.circle(
        hw(attempt ^ ord('i')),
        trace[45],
        line_color=Category10_10[hw(attempt ^ ord('i'))],
    )

show(p)